<a href="https://colab.research.google.com/github/jai-sundaram/ml_tutorial/blob/main/my_data_preprocessing_tools.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data Preprocessing Tools

## Importing the libraries

In [ ]:
##numpy helps us work with arrays
import numpy as np
#matplotlib helps us plot numerous charts, particularly pylpot which allows us to plot nice charts
import matplotlib.pyplot as plt
#pandas helps import the dataset, create the matrix, and independent variable vector
import pandas as pd

## Importing the dataset

In [ ]:
#reading the values of the data set and creating a data frame
dataset = pd.read_csv('Data.csv')
#creating two additional entities
#creating a matrix of features and a dependent variable vector

#the features are the columns you are using to predict the dependent variable
#the dependent variable is the last column, the thing you are predicting
#in other terms, the features are the independent variables

##in this case, the country, age, and salary are the features/dependent variabels
##the fact if they purchased or not is the dependent variable

#formatting wise, most datasets will have the features/dependent variables in the first few columns, and the dependent variable in the last column

## Taking care of missing data

## Encoding categorical data

### Encoding the Independent Variable

### Encoding the Dependent Variable

## Splitting the dataset into the Training set and Test set

## Feature Scaling